
**IMPORTANT**: 
> Incase you add lines later in the docker file that requires apt-get update to be run before, make sure update added just before that line. Otherwise,if update occered earlier in the file, it will be in cache and it wont be executed when Dockerfile had been changed.

## 1)  Sample Docker Setup for using webcam

Here is an example of setting up docker container from **Ubuntu 16.04** base image

See **docker-webcam/** for detailed example

**Dockerfile**

```dockerfile
FROM ubuntu:16.04

RUN apt-get update

# For bash, and auto-complete on command line l
# -y specifies we confirm the installation. Otherwise, the code will hang up
RUN apt-get -y install build-essential 

# To find OS version (lsb_release -a)
RUN apt-get -y install lsb-release 

# python3.5 gets installed
RUN apt-get -y install python3-pip

#To view files
RUN apt-get -y install vim 

#Install python dependencies
WORKDIR /home/python-requirements
COPY python-requirements/requirements.txt .
RUN pip3 install -r requirements.txt

WORKDIR /home

CMD ["bash"] 
```

**Build**

```bash
docker build -t senesence/docker-webcam .
```

**Run**

> The device should have been listed before running the docker 
```bash
docker run -it --device /dev/video0 ...
```
>> In case it gets complicated to find devices, use **--privileged** and map all devices 
```bash
docker run -it --privileged -v /dev:/dev ...
```

> **To find the device folder in your machine (command line)**
```bash
sudo apt-get install v4l-utils
v4l2-ctl --list-devices
```
>> **OUTPUT** : 
Webcam C170 (usb-0000:00:14.0-4):
	/dev/video0


```bash
docker run -it -p 0.0.0.0:8000:8888 -p 0.0.0.0:6005:6006 -v ${1:-$PWD}:/home  --device /dev/video0 --runtime=nvidia senesence/docker-webcam
```

**Run jupyter notebook inside container**

Does not work without --ip=0.0.0.0
```bash
jupyter-notebook --ip=0.0.0.0 --allow-root
```

## 2)  Sample Docker Setup for using audio devices

See **docker-audio/** for detailed example

### Set up audio drivers

We use **pulseaudio server as default** [Reference](https://askubuntu.com/questions/972510/how-to-set-alsa-default-device-to-pulseaudio-sound-server-on-docker)

> **Install audio drivers and dependencies**
```bash
RUN apt-get -y install libportaudio2
RUN apt-get -y install alsa-base 
RUN apt-get -y install alsa-utils #for playing sound files with aplay 
RUN apt-get -y install pulseaudio
```

> **Set PULSE_SERVER** environment variable, **mount** the corresponding directory. Mount the **cookie directory** as well, without which we get alsa permission denied error
```bash
echo $XDG_RUNTIME_DIR
'/run/user/1000'
docker run -it \
        -e PULSE_SERVER=unix:${XDG_RUNTIME_DIR}/pulse/native \ #Set environment variable 
        -v ${XDG_RUNTIME_DIR}/pulse/native:${XDG_RUNTIME_DIR}/pulse/native \ #Mount directory
        -v ~/.config/pulse/cookie:/root/.config/pulse/cookie #Set up cookie 
```

> **Check setup** (in the container)
```bash
aplay -L | head -n9
```
>> **Output** :
```bash
default
    Playback/recording through the PulseAudio sound server
    ...
```

> Check if the speaker works when you play an audio file
```bash
aplay sample.wav
```

**Dockerfile**

```dockerfile
FROM ubuntu:16.04

RUN apt-get update
RUN apt-get -y install build-essential lsb-release #For bash, lsb_release -a
RUN apt-get -y install python3-pip
RUN apt-get -y install vim 

#Incase you add this line later, make sure update is run. Sometimes, update will be in cache and it wont be executed
RUN apt-get -y install libportaudio2
RUN apt-get -y install alsa-base alsa-utils pulseaudio

WORKDIR /home/python-requirements
COPY python-requirements/requirements.txt .
RUN pip3 install -r requirements.txt

WORKDIR /home

CMD ["bash"]
```

**Run**

> Make sure **--device /dev/snd** is added

```bash
docker run -it -p 8000:8888 -p 0.0.0.0:6005:6006 -v ${1:-$PWD}:/home --device /dev/snd  -e PULSE_SERVER=unix:${XDG_RUNTIME_DIR}/pulse/native -v ${XDG_RUNTIME_DIR}/pulse/native:${XDG_RUNTIME_DIR}/pulse/native -v ~/.config/pulse/cookie:/root/.config/pulse/cookie  --runtime=nvidia senesence/docker-audio
```    